Can I change the text inside the raw text file?

In [ ]:
# Install package to be able to save keras weights
pip install h5py

In [ ]:
pip install nibabel

In [ ]:
# Mount google drive
from google.colab import drive
import os
drive.mount('my_drive', force_remount=True)

In [ ]:
from keras.callbacks import Callback
import json
import psutil
import os

def print_memory_use():
    '''
    Function which prints current python memory usage
    '''
    process = psutil.Process(os.getpid())
    print(process.memory_info().rss/1e9)

# What value maps to what class
mapping = {
    0: "Null class",
    1: "Necrotic and non-enhancing tumor core",
    2: "Edema",
    4: "GD-enhancing tumor"
}

# fixme: skulle gå att göra bättre igenom att skicka med en tex tuple med titlarna
# och returnera ett matplotlib-objekt istället för då hade man inte behövt ha olika
# funktioner för "plot_modalities" och "plt_OHE" och också kunna ha två stycken figurer med 
# 2*2 subplots i en cell.
def plot_modalities(x):
    # Make sure input data is of correct shape
    assert x.shape == (240, 240, 4), 'Shape of input data is incorrect'
    
    plt.subplot('221')
    plt.imshow(x[:,:,0])
    plt.axis('off')
    plt.title('T1')
    plt.colorbar()

    plt.subplot('222')
    plt.imshow(x[:,:,1])
    plt.axis('off')
    plt.title('T1ce')
    plt.colorbar()

    plt.subplot('223')
    plt.imshow(x[:,:,2])
    plt.axis('off')
    plt.title('T2')
    plt.colorbar()

    plt.subplot('224')
    plt.imshow(x[:,:,3])
    plt.axis('off')
    plt.title('FLAIR')
    plt.colorbar()

def plot_OHE(y):
    # Make sure input data is of correct shape
    assert y.shape == (240, 240, 4), 'Shape of input data is incorrect'
    
    plt.subplot('221')
    plt.imshow(y[:,:,0])
    plt.axis('off')
    plt.title('Null')
    plt.colorbar()

    plt.subplot('222')
    plt.imshow(y[:,:,1])
    plt.axis('off')
    plt.title('"Necrotic and non-enhancing tumor core"')
    plt.colorbar()

    plt.subplot('223')
    plt.imshow(y[:,:,2])
    plt.axis('off')
    plt.title('Edema')
    plt.colorbar()

    plt.subplot('224')
    plt.imshow(y[:,:,3])
    plt.axis('off')
    plt.title('GD-enhancing tumor')
    plt.colorbar()

def shift_and_scale(x):
    assert len(x.shape) == 2, 'The input must be 2 dimensional'
    #assert np.std(x) != 0, 'Cant divide by zero'
    result = x - np.mean(x)
    
    # This is a really ugly hack
    if np.std(x) == 0:
      result /= 1
    else:
      result /= np.std(x)
    return result

def OHE(Y, mapping):
    '''
    Takes in a picture as a matrix with labels and returns a one hot encoded tensor
    
    Parameters:
    Y is the picture
    Mapping is what value corresponds to what label
    
    Returns:
    A tensor with a channel for each label.
    '''
    shape = Y.shape
    labels = mapping.keys()
    one_hot_enc = np.zeros(list(shape) + [len(labels)])
    
    for i, label in enumerate(labels):
        temp = np.zeros(shape)
        ind = Y == label
        temp[ind] = 1
        one_hot_enc[:, :, i] = temp
    return one_hot_enc

#fixme: I don't know if providing this mapping is necessary
# probably could be provided inside function instead.
def OHE_uncoding(y, mapping):
    result = np.argmax(y, axis=2)
    labels = mapping.keys()
    temp = np.zeros(result.shape)
    for i, label in enumerate(labels):
        ind = result == i
        temp[ind] = label
    return temp

def IoU_wholeTumor(y_true, y_pred):
    values = np.array([0., 1.])
    unique_y_pred = np.unique(y_pred)
    unique_y_true = np.unique(y_true)
    assert np.array_equal(y_pred.shape, y_true.shape), 'Prediction and ground truth must have same shape'
    assert np.array_equal(values, unique_y_pred), 'yhat and y must be one hot encodings'
    assert np.array_equal(values, unique_y_true), 'yhat and y must be one hot encodings'
    
    
    y_pred[:,:,0] = np.logical_not(y_pred[:,:,0]) 
    y_true[:,:,0] = np.logical_not(y_true[:,:,0])
    
    intersection = np.logical_and(y_pred[:,:,0], y_true[:,:,0])
    union = np.logical_or(y_true[:,:,0], y_pred[:,:,0])
    
    size_int = np.count_nonzero(intersection)
    size_uni = np.count_nonzero(union)
    
    return size_int/size_uni

def dice_coefficient(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(K.abs(y_true_f * y_pred_f), axis=-1)
    return (2. * intersection) / (
        K.sum(K.square(y_true_f), -1) + K.sum(K.square(y_pred_f), -1) + 1e-8)
def reset_config(config, config_path=None, weights_path=None):
    new_config = config
    if weights_path:
        assert type(weights_path) == str, 'The weight path must be a string'
        new_config['weights_path'] = weights_path
    if config_path:
        assert type(config_path) == str, 'The config path must be a string'
        new_config['config_path'] = config_path
    new_config['history']['training_samples_used'] = 0
    new_config['history']['loss'] = []
    new_config['history']['val_loss'] = []
    new_config['keep_training'] = False

class CallbackJSON(Callback):
    """ CallbackJSON descends from Callback
        and is used to write the number of training samples that the model has been trained on
        and the loss for a epoch
    """
    def __init__(self, config):
        """Save params in constructor
        config: Is a dictionary loaded from a JSON file which is used to keep track of training
        """
        self.config = config
        self.config_path = config['config_path']

    def on_epoch_end(self, epoch, logs):
        """
        Updates the history of the config dict and saves it to a file
        """
        # How many effective training samples have been used
        self.config['history']['training_samples_used'] += self.config['samples_used']
        
        # Logs the loss of the current epoch
        self.config['history']['loss'].append(logs['loss'])
        #fixme: add the same code but for "val_loss"
        #self.config['history']['val_loss'].append(logs['val_loss'])
        
        print_memory_use()
        # Save new config file
        with open(self.config_path, "w") as f:
            f.write(json.dumps(self.config))

print('Finished')
print_memory_use()

First part gets all paths to the different modalities and second part loads them into numpy arrays

In [ ]:
import nibabel as nib
import matplotlib.pyplot as plt
import numpy as np
import glob
from os import listdir
import os

# Code snippet to fix that colab notebook and local notebook access data
# through different paths
var = os.uname()
run_on_colab = var[0] == "Linux"
if run_on_colab:
    wild_t1 = "/content/my_drive/My Drive/EXJOBB/MICCAI_BraTS_2019_Data_Training/*/*/*_t1.nii.gz"
    wild_t1ce = "/content/my_drive/My Drive/EXJOBB/MICCAI_BraTS_2019_Data_Training/*/*/*_t1ce.nii.gz"
    wild_t2 = "/content/my_drive/My Drive/EXJOBB/MICCAI_BraTS_2019_Data_Training/*/*/*_t2.nii.gz"
    wild_flair = "/content/my_drive/My Drive/EXJOBB/MICCAI_BraTS_2019_Data_Training/*/*/*_flair.nii.gz"
    wild_gt = "/content/my_drive/My Drive/EXJOBB/MICCAI_BraTS_2019_Data_Training/*/*/*_seg.nii.gz"
else:
    wild_t1 = "MICCAI_BraTS_2019_Data_Training/*/*/*_t1.nii.gz"
    wild_t1ce = "MICCAI_BraTS_2019_Data_Training/*/*/*_t1ce.nii.gz"
    wild_t2 = "MICCAI_BraTS_2019_Data_Training/*/*/*_t2.nii.gz"
    wild_flair = "MICCAI_BraTS_2019_Data_Training/*/*/*_flair.nii.gz"
    wild_gt = "MICCAI_BraTS_2019_Data_Training/*/*/*_seg.nii.gz"

t1_paths = glob.glob(wild_t1)
t1ce_paths = glob.glob(wild_t1ce)
t2_paths = glob.glob(wild_t2)
flair_paths = glob.glob(wild_flair)
gt_paths = glob.glob(wild_flair)

num_patients = 5
num_slices = 155
org_shape = (4, 240, 240, 155)
uncorrected_data = np.zeros((4, 240, 240, num_patients*num_slices))
uncorrected_OHE_labels = np.zeros((num_patients*num_slices, 240, 240, 4))

for i in range(num_patients):
  print('Patient: ' + str(i))
  path_t1 = t1_paths[i]
  path_t1ce = t1ce_paths[i]
  path_t2 = t2_paths[i]
  path_flair = flair_paths[i]
  path_gt = gt_paths[i]

  img_t1 = nib.load(path_t1)
  img_t1ce = nib.load(path_t1ce)
  img_t2 = nib.load(path_t2)
  img_flair = nib.load(path_flair)
  img_gt = nib.load(path_gt)

  img_t1 = img_t1.get_fdata()
  img_t1ce = img_t1ce.get_fdata()
  img_t2 = img_t2.get_fdata()
  img_flair = img_flair.get_fdata()
  img_gt = img_gt.get_fdata()

  # I have here chosen to do shift and scale per image, 
  # which is not the only way to do normalization.
  for j in range(img_t1.shape[2]):
      # shift and scale data
      img_t1[:, :, j] = shift_and_scale(img_t1[:, :, j])
      img_t1ce[:, :, j] = shift_and_scale(img_t1ce[:, :, j])
      img_t2[:, :, j] = shift_and_scale(img_t2[:, :, j])
      img_flair[:, :, j] = shift_and_scale(img_flair[:, :, j])

  assert not np.any(np.isnan(img_t1)), 'Inputs contain nans'
  assert not np.any(np.isnan(img_t1ce)), 'Inputs contain nans'
  assert not np.any(np.isnan(img_t2)), 'Inputs contain nans'
  assert not np.any(np.isnan(img_flair)), 'Inputs contain nans'

  uncorrected_data[0, :, :, i*num_slices:(i+1)*num_slices] = img_t1
  uncorrected_data[1, :, :, i*num_slices:(i+1)*num_slices] = img_t1ce
  uncorrected_data[2, :, :, i*num_slices:(i+1)*num_slices] = img_t2
  uncorrected_data[3, :, :, i*num_slices:(i+1)*num_slices] = img_flair
  
  for j in range(num_slices):
      uncorrected_OHE_labels[j, :, :, :] = OHE(img_gt[:, :, j], mapping)

# The last axis will become the first axis
uncorrected_data = np.moveaxis(uncorrected_data, -1, 0)
uncorrected_data = np.moveaxis(uncorrected_data, 1, 3)

print('Finished')
print_memory_use()

Load the next patient and use as validation data

In [ ]:
validation_index = num_patients

validation_data = np.zeros((4, 240, 240, num_slices))
validation_OHE_labels = np.zeros((num_slices, 240, 240, 4))

# Pick one patient for validation
path_t1 = t1_paths[validation_index]
path_t1ce = t1ce_paths[validation_index]
path_t2 = t2_paths[validation_index]
path_flair = flair_paths[validation_index]
path_gt = gt_paths[validation_index]

img_t1 = nib.load(path_t1)
img_t1ce = nib.load(path_t1ce)
img_t2 = nib.load(path_t2)
img_flair = nib.load(path_flair)
img_gt = nib.load(path_gt)

img_t1 = img_t1.get_fdata()
img_t1ce = img_t1ce.get_fdata()
img_t2 = img_t2.get_fdata()
img_flair = img_flair.get_fdata()
img_gt = img_gt.get_fdata()

# I have here chosen to do shift and scale per image, 
# which is not the only way to do normalization.
for j in range(img_t1.shape[2]):
    # shift and scale data
    img_t1[:, :, j] = shift_and_scale(img_t1[:, :, j])
    img_t1ce[:, :, j] = shift_and_scale(img_t1ce[:, :, j])
    img_t2[:, :, j] = shift_and_scale(img_t2[:, :, j])
    img_flair[:, :, j] = shift_and_scale(img_flair[:, :, j])

assert not np.any(np.isnan(img_t1)), 'Inputs contain nans'
assert not np.any(np.isnan(img_t1ce)), 'Inputs contain nans'
assert not np.any(np.isnan(img_t2)), 'Inputs contain nans'
assert not np.any(np.isnan(img_flair)), 'Inputs contain nans'

validation_data[0, :, :, :] = img_t1
validation_data[1, :, :, :] = img_t1ce
validation_data[2, :, :, :] = img_t2
validation_data[3, :, :, :] = img_flair

for j in range(num_slices):
    validation_OHE_labels[j, :, :, :] = OHE(img_gt[:, :, j], mapping)

# The last axis will become the first axis
validation_data = np.moveaxis(validation_data, -1, 0)
validation_data = np.moveaxis(validation_data, 1, 3)

validation_OHE_labels = validation_OHE_labels.reshape(num_slices,-1,4)

print('Finished')
print_memory_use()

In [ ]:
# This will show a slice of a patient
ind = 70
patient = uncorrected_data[ind, :, :, :]
print(patient.shape)
plot_modalities(patient)

for i in range(4):
  print("Max:", np.max(patient[:,:,i]), "Min:", np.min(patient[:,:,i]), sep=" " )

In [ ]:
import numpy as np
import os
import skimage.io as io
import skimage.transform as trans
import numpy as np
from keras.models import *
from keras.layers import *
from keras.activations import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as keras

def unet(pretrained_weights = None, input_size = (256, 256, 1), num_classes=1, learning_rate=1e-3):
  inputs = Input(input_size)
  conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
  conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
  pool1 = MaxPooling2D(pool_size = (2, 2))(conv1)
  
  conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
  conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
  pool2 = MaxPooling2D(pool_size = (2, 2))(conv2)
  
  conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
  conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
  pool3 = MaxPooling2D(pool_size = (2, 2))(conv3)
  
  conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
  conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
  drop4 = Dropout(0.5)(conv4)
  pool4 = MaxPooling2D(pool_size = (2, 2))(drop4)
  
  conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
  conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
  drop5 = Dropout(0.5)(conv5)
  
  up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2, 2))(drop5))
  merge6 = concatenate([drop4, up6], axis = 3)
  conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
  conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)
  
  up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2, 2))(conv6))
  merge7 = concatenate([conv3, up7], axis = 3)
  conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
  conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)
  
  up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2, 2))(conv7))
  merge8 = concatenate([conv2, up8], axis = 3)
  conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
  conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)
  
  up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2, 2))(conv8))
  merge9 = concatenate([conv1, up9], axis = 3)
  conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
  conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
  conv9 = Conv2D(num_classes, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
  
  reshape = Reshape((num_classes, input_size[0] * input_size[1]), input_shape = (num_classes, input_size[0], input_size[1]))(conv9)
  permute = Permute((2, 1))(reshape)
  activation = Softmax(axis=-1)(permute)
  
  model = Model(input = inputs, output = activation)
  model.compile(optimizer = Adam(lr=learning_rate), loss = 'categorical_crossentropy', metrics=[dice_coefficient])
  if (pretrained_weights):
    model.load_weights(pretrained_weights)
  return model

# New main training cell

In [ ]:
# Load config file to session here
if run_on_colab:
    config_path = "/content/my_drive/My Drive/EXJOBB/training_sessions/Session_1/config_1.json"
else:
    config_path = "training_session_2/config_2.json"
with open(config_path, 'r') as config_file:
    config = json.load(config_file)
print(config)

#print(reset_config.__code__.co_varnames)

#reset_config(config, config_path = config_path, weights_path="training_session_2/weights_2.h5")
print(config)


In [ ]:
# The path to where to save weights and initialize ModelCheckpoint
weights_path = config['weights_path']
from keras.callbacks import ModelCheckpoint
MyModelCheckPoint = ModelCheckpoint(weights_path, verbose=0, save_weights_only=True)

print(config['keep_training'])

if config['keep_training'] == True:
    # Keep training on the old weights
    my_unet = unet(input_size = (240, 240, 4), num_classes = 4)
    my_unet.load_weights(weights_path)
else:
    # Initialize network
    my_unet = unet(input_size = (240, 240, 4), num_classes = 4)
    config['keep_training'] = True
    
samples_used = 155
# i = 70 has all labels present in the image
n = uncorrected_data.shape[0]
#i = np.random.randint(n, size=[samples_used,])
i = [70]
x = uncorrected_data[i, :, :, :]
y = uncorrected_OHE_labels[i, :, :, :]
y = y.reshape(len(i), -1, 4)

print_memory_use()

#del uncorrected_data
#del uncorrected_OHE_labels

print_memory_use()

assert not np.any(np.isnan(x)), 'Input contain nans'

# Returns an object with accuracy and loss
history = my_unet.fit(x=x, 
                      y=y, 
                      batch_size=None,
                      epochs=500, 
                      verbose=1, 
                      callbacks=[CallbackJSON(config=config), MyModelCheckPoint],
                      validation_split=0.0, 
                      #validation_data=(validation_data, validation_OHE_labels), 
                      shuffle=True, 
                      class_weight=None, 
                      sample_weight=None, 
                      initial_epoch=0, 
                      steps_per_epoch=None, 
                      validation_steps=None, 
                      validation_freq=1)


In [ ]:
loss = config['history']['loss']

plt.plot(loss)

In [ ]:
my_unet = unet(input_size = (240, 240, 4), num_classes = 4)
my_unet.load_weights(config['weights_path'])

yhat = my_unet.predict(uncorrected_data[70, :, :, :].reshape(1, 240, 240, 4))

plot_OHE(yhat.reshape(240, 240, 4))